In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
    sum(case when CRIM is null then 1 else 0 end) as CRIM_null,
    sum(case when ZN is null then 1 else 0 end) as ZN_null,
    sum(case when INDUS is null then 1 else 0 end) as INDUS_null,
    sum(case when CHAS is null then 1 else 0 end) as CHAS_null,
    sum(case when NOX is null then 1 else 0 end) as NOX_null
    FROM boston
    ''',
    con,
)

,CRIM_null,ZN_null,INDUS_null,CHAS_null,NOX_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT
    count(distinct CRIM) as CRIM_uniq,
    count(distinct ZN) as ZN_uniq,
    count(distinct INDUS) as INDUS_uniq,
    count(distinct CHAS) as CHAS_uniq,
    count(distinct NOX) as NOX_uniq
    FROM boston
    ''',
    con,
)

,CRIM_uniq,ZN_uniq,INDUS_uniq,CHAS_uniq,NOX_uniq
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT distinct
    case when med_CRIM=min_CRIM THEN 'CRIM' end as CRIM,
    case when med_ZN=min_ZN THEN 'ZN' end as ZN,
    case when med_INDUS=min_INDUS THEN 'INDUS' end as INDUS,
    case when med_CHAS=min_CHAS THEN 'CHAS' end as CHAS,
    case when med_NOX=min_NOX THEN 'NOX' end as NOX
    FROM 
    (SELECT avg(CRIM) as med_CRIM
    FROM (
          SELECT CRIM
          FROM boston
          ORDER BY CRIM asc
          LIMIT 2 - (SELECT count(*) FROM boston)%2
          OFFSET (SELECT (count(*)-1)/2 FROM boston)))
    CROSS JOIN 
    (SELECT avg(ZN) as med_ZN
    FROM (
          SELECT ZN
          FROM boston
          ORDER BY ZN asc
          LIMIT 2 - (SELECT count(*) FROM boston)%2
          OFFSET (SELECT (count(*)-1)/2 FROM boston)))
    CROSS JOIN
    (SELECT avg(INDUS) as med_INDUS
    FROM (
          SELECT INDUS
          FROM boston
          ORDER BY INDUS asc
          LIMIT 2 - (SELECT count(*) FROM boston)%2
          OFFSET (SELECT (count(*)-1)/2 FROM boston)))
    CROSS JOIN
    (SELECT avg(CHAS) as med_CHAS
    FROM (
          SELECT CHAS
          FROM boston
          ORDER BY CHAS asc
          LIMIT 2 - (SELECT count(*) FROM boston)%2
          OFFSET (SELECT (count(*)-1)/2 FROM boston)))
    CROSS JOIN
    (SELECT avg(NOX) as med_NOX
    FROM (
          SELECT NOX
          FROM boston
          ORDER BY NOX asc
          LIMIT 2 - (SELECT count(*) FROM boston)%2
          OFFSET (SELECT (count(*)-1)/2 FROM boston)))
    CROSS JOIN 
    (SELECT min(CRIM) as min_CRIM
    , min(ZN) as min_ZN
    , min(INDUS) as min_INDUS
    , min(CHAS) as min_CHAS
    , min(NOX) as min_NOX
    FROM boston
    )
    WHERE med_ZN= min_ZN
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,None,ZN,None,CHAS,None


CHAS - 50% не граничит с рекой. ZN - 50% участков свыше 25,000 sq.ft, не используются под жилую застройку.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql(
    '''
    WITH help_table AS (
         SELECT * FROM
         (SELECT 
          RM
          , MEDV
          , row_number() over (order by MEDV asc) as rn_low
          FROM boston)
          WHERE rn_low <= 300),
         max_houses AS (
          SELECT avg(RM) as RM_high, avg(MEDV) as MEDV_high 
          FROM boston
          WHERE MEDV = (SELECT max(MEDV) FROM boston)
         )
    SELECT *
    , (diff_price /
        last_value(diff_price) over (ORDER BY diff_price desc ROWS BETWEEN CURRENT
           ROW AND 1 FOLLOWING) - 1)*100 as f
    FROM (     
            SELECT 
            25 as houses,
            RM_high,
            avg(RM) as RM_low,
            RM_high - avg(RM) as diff,
            MEDV_high - avg(MEDV) as diff_price
            FROM help_table t1
            CROSS JOIN max_houses t2
            WHERE rn_low <= 25
        UNION ALL
            SELECT 
            50,
            RM_high,
            avg(RM) as RM_low,
            RM_high - avg(RM) as diff,
            MEDV_high - avg(MEDV) as diff_price
            FROM help_table t1
            CROSS JOIN max_houses t2
            WHERE rn_low <= 50
        UNION ALL
            SELECT 
            100,
            RM_high,
            avg(RM) as RM_low,
            RM_high - avg(RM) as diff,
            MEDV_high - avg(MEDV) as diff_price
            FROM help_table t1
            CROSS JOIN max_houses t2
            WHERE rn_low <= 100
        UNION ALL
            SELECT 
            200,
            RM_high,
            avg(RM) as RM_low,
            RM_high - avg(RM) as diff,
            MEDV_high - avg(MEDV) as diff_price
            FROM help_table t1
            CROSS JOIN max_houses t2
            WHERE rn_low <= 200
        UNION ALL
            SELECT 
            300,
            RM_high,
            avg(RM) as RM_low,
            RM_high - avg(RM) as diff,
            MEDV_high - avg(MEDV) as diff_price
            FROM help_table t1
            CROSS JOIN max_houses t2
            WHERE rn_low <= 300)
    ''',
    con,
)

,houses,RM_high,RM_low,diff,diff_price,f
0,25,7.484,5.747840,1.736160,42.152000,4.465923
1,50,7.484,5.753240,1.730760,40.350000,5.675301
2,100,7.484,5.887120,1.596880,38.183000,8.605561
3,200,7.484,5.911705,1.572295,35.157500,6.304867
4,300,7.484,5.972227,1.511773,33.072333,0.000000


Да, вляияет, чем дороже дома, тем менее они отличаются от дорогих домов по количеству комнат. Разница в стоимости колеблется между дорогими и дешёвыми домами колеблется в разрезе 4-8%.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
WITH rank_r AS (
         SELECT LSTAT as LSTAT_among_r, MEDV as MEDV_among_r, rank() over (order by LSTAT) as rank_r
         FROM (
          SELECT *
          FROM boston
          WHERE MEDV = (SELECT max(MEDV) FROM boston))),
    rank_a AS (
         SELECT LSTAT as LSTAT_among_a, MEDV as MEDV_among_a, rank() over (order by LSTAT) as rank_a
         FROM boston)
    SELECT LSTAT_among_a as LSTAT, rank_a, rank_r, MEDV_among_r as MEDV
    FROM rank_a t1
    INNER JOIN rank_r t2 ON t1.LSTAT_among_a=t2.LSTAT_among_r AND t1.MEDV_among_a=t2.MEDV_among_r
    ''',
    con,
)

,LSTAT,rank_a,rank_r,MEDV
0,1.73,1,1,50.0
1,1.92,2,2,50.0
2,2.88,6,3,50.0
3,2.96,8,4,50.0
4,2.97,9,5,50.0
5,3.16,15,6,50.0
6,3.26,17,7,50.0
7,3.32,18,8,50.0
8,3.70,26,9,50.0
9,3.73,27,10,50.0


Самые дорогие дома не гарантируют более престижный район.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
    case when CHAS=0 then 'граничит' else 'не граничит' end as chas,
    avg(MEDV) as avg_MEDV
    FROM boston
    GROUP BY CHAS
    ''',
    con,
)

,chas,avg_MEDV
0,граничит,22.093843
1,не граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT max(CRIM), max(ZN), max(INDUS), max(CHAS), max(NOX)
    FROM (
    SELECT
    CASE WHEN first_value(avg_CRIM) over (ORDER BY CHAS asc) > avg_CRIM THEN 'CRIM' END as CRIM,
    CASE WHEN first_value(avg_ZN) over (ORDER BY CHAS asc) > avg_ZN THEN 'ZN' END as ZN,
    CASE WHEN first_value(avg_INDUS) over (ORDER BY CHAS asc) > avg_INDUS THEN 'INDUS' END as INDUS,
    CASE WHEN first_value(avg_CHAS) over (ORDER BY CHAS asc) > avg_CHAS THEN 'CHAS' END as CHAS,
    CASE WHEN first_value(avg_NOX) over (ORDER BY CHAS asc) > avg_NOX THEN 'NOX' END as NOX
    FROM (
    SELECT CHAS, avg(CRIM) as avg_CRIM, avg(ZN) as avg_ZN, avg(INDUS) as avg_INDUS, avg(CHAS) as avg_CHAS, avg(NOX) as avg_NOX
    FROM boston
    GROUP BY CHAS))
    ''',
    con,
)

,max(CRIM),max(ZN),max(INDUS),max(CHAS),max(NOX)
0,CRIM,ZN,None,None,None


Рядом с реками уровень преступности выше и в среднем доля больше земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT
    perc_indus, max(perc_nox), min(perc_nox)
    FROM
    (SELECT
    INDUS
    , NOX
    , NTILE(10) OVER (ORDER BY INDUS asc) * 10 as perc_indus
    , NTILE(10) OVER (ORDER BY NOX asc) * 10 as perc_nox
    FROM boston)
    GROUP BY perc_indus
    ORDER BY perc_indus asc
    ''',
    con,
)

,perc_indus,max(perc_nox),min(perc_nox)
0,10,50,10
1,20,80,10
2,30,50,10
3,40,50,10
4,50,70,40
5,60,70,10
6,70,80,20
7,80,90,80
8,90,100,70
9,100,100,70


При значительной индустриализации, концентрация оксидов азота также находится на высоком уровне.

In [15]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
